In [ ]:
from platform import python_version

print(python_version())

In [ ]:
######################################## Lire les données ########################################################"
import pandas as pd
import scipy

##############################################
VERBOSE_VALUE = 0 # Afficher les logs d'exécution de GridSearch
TEST_SIZE_VALUE = 0.1 # Taux du jeu de données à utiliser dans l'étape de test
CROSS_VALID = 5 # Nombre de K pour la validation croisée
###############################################


data = pd.read_csv("/home/manel/Bureau/BD/donnee+pluvio/smv+vdp-pluvio/donnee-sans-na/smv-vdp/seine.csv")
### Supprimer toutes les valeurs vides
data = data.fillna(0) 

X = data.drop(["Ecoli","EI"],axis=1)
y = data["Ecoli"]

print(X)
print(y)


In [ ]:

######################################## informations sur les données ########################################################"
print("  * * * * * * * I)  Dataframe information * * * * * * *  ")
print(" 1) Statistics  _  describe ()  ")
print(data.describe())
print("______________________________________")
print(" 2) Informaton (Column, Non-Null, Count, Dtype)  _  info ()  ")
print(data.info())

print("TYPE === ",type(data))


In [ ]:
# Analyse descriptive des données
#! pip install pandas_profiling
from pandas_profiling import ProfileReport
prof = ProfileReport(data, title='Analyse du jeu de la Seine', html={'style':{'full_width':True}})
prof.to_notebook_iframe()

In [ ]:

##############################################################################################"
######################################## Split des données  #########################################"
##############################################################################################"

# Diviser les données en entrainement et test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE_VALUE, random_state=0)
# La division est faite de manière aléatoire en divisant les données en entrainement et en test. 
# Le random state permet d'avoir une graine pour la reproductibilité. 
# Le changement de la valeur modifie la division appliquée aux données afin d'obtenir des jeux d'entrainements et de tests différents. 

# Supprimez la colonne date du jeu d'entrainement
X_train = X_train.drop(["date"],axis=1)

# Créer le test de données à exporter
DataToExport = X_test
df_Ytest = pd.DataFrame(data=y_test.values, columns=['list'])

pieces = {'x': DataToExport, 'y': df_Ytest}

DataToExport = pd.concat(pieces)

# Imprimer les données à exporter
print(DataToExport)


print("  * * * * * * * -- Export -- * * * * * * *  ")
X_test.to_csv ('data_Test.csv', index = False, header=True, sep=";")
DataToExport = pd.read_csv("data_Test.csv", sep=";")
print(type(DataToExport))
df_Ytest = pd.DataFrame(data=y_test.values)
DataToExport["Ecolireal"] = df_Ytest
print("  * * * * * * * -- DATA TEST building -- OK -- * * * * * * *  ")
X_test = X_test.drop(["date"],axis=1)

### Standardisation des données

La standardisation permet la mise à l'échelle avant l'entraînement. Elle sera effectuée sur les données d'entraînement afin que les modèles n'aient pas accès aux valeurs des données de test.
Standardisation par la moyenne : Soustraire la moyenne et diviser par l'écart-type pour chaque caractéristique.

In [ ]:
# Calculer les valeurs de la moyenne et de l'écart-type
X_train, X_test, y_train, y_test
train_mean = X_train.mean()
train_std = X_train.std()
y_mean = y_train.mean()
y_std = y_train.std()

X_train = (X_train - train_mean) / train_std
X_test = (X_test - train_mean) / train_std
y_train = (y_train - y_mean) / y_std
y_test = (y_test - y_mean) / y_std

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
# Détermination de l'écart-type après standardisation. 
import numpy as np
sd=np.std(y_test)
print(sd)

In [ ]:
##### ---------------------- Imports ----------------------------------------------------------------------
from sklearn.model_selection import GridSearchCV 
#Créer la grille de paramètres basée sur les résultats de la recherche aléatoire.
import numpy as np


##############################################################################################"
######################################## MODEL KNN Split  ####################################"
##############################################################################################"


## --- Local Params
modelName = "KNN"
MethodID = "A"

print( " ################# ------------- ########################## ")
print( "Model " ,modelName," is runing ........ !  ")
from sklearn.neighbors import KNeighborsRegressor
param_gridKNN = {'n_neighbors': np.arange(1, 30, 2),
              'weights': ['uniform', 'distance']
         }


# Sélection du meilleur modèle
modelKNN =  KNeighborsRegressor()
best_model_searchKNN = GridSearchCV(estimator = modelKNN, param_grid = param_gridKNN, cv = CROSS_VALID,
                           verbose = VERBOSE_VALUE)
best_model_searchKNN.fit(X_train,y_train)


#  Montrer quel est le meilleur modèle
best_grid = best_model_searchKNN.best_estimator_
print("  ------------------------------------  ")
print (MethodID+"-1) BEST Configuration ("+modelName+") is  ==== ",best_grid )
print("  ------------------------------------  ")


# prédire les valeurs en utilisant le meilleur modèle
Y_pred_KNN = best_model_searchKNN.predict(X_test)
Y=y_test
num_data = X.shape[0]


# Erreurs pour l'évaluation des performances
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

mse = mean_squared_error(Y,Y_pred_KNN)
rmse = math.sqrt(mse)
rse = math.sqrt(mse/(num_data-2))
mae=mean_absolute_error(Y,Y_pred_KNN)

print(MethodID+"-2) Evaluation  "+modelName+" Results : ")
print("--> RMSE (("+modelName+"))=",rmse)
print("--> MAE (("+modelName+"))=",mae)
r = scipy.stats.pearsonr(Y,Y_pred_KNN)
print("--> Pearson Correlation (("+modelName+"))=",r)


# Exporter le modèle
import pickle
filename = 'bestModel'+modelName+'-pluvio-vdp-nrml-Ecoli.pickle'
pickle.dump(best_model_searchKNN, open(filename, 'wb'))
print("  ------------------------------------  ")
print( MethodID+"-3) Model " ,modelName," is generated and exported -- OK --  ")
print( " ################# ------------- ########################## ")


# Destandardisation des valeurs prédites
Y_pred_KNN = (Y_pred_KNN*y_std)+ y_mean
DataToExport["KNN_Ecoli_pred"] = Y_pred_KNN


In [ ]:
##############################################################################################"
######################################## MODEL RF  Split  ####################################"
##############################################################################################"

from sklearn.ensemble import RandomForestRegressor
## --- Local Params
modelName = "RF"
MethodID = "B"

print( " ################# ------------- ########################## ")
print( "Model " ,modelName," is runing ........ !  ")
from sklearn.neighbors import KNeighborsRegressor

param_gridRF = {
    'bootstrap': [True],
    'max_features': [2,3,4,5,6,7,8],
    'n_estimators': [10, 50, 200]
}


# Sélection du meilleur modèle
modelRF =  RandomForestRegressor()
best_model_searchRF = GridSearchCV(estimator = modelRF, param_grid = param_gridRF, cv = CROSS_VALID,
                           verbose = VERBOSE_VALUE)
best_model_searchRF.fit(X_train,y_train)


# Montrer quel est le meilleur modèle
best_grid = best_model_searchRF.best_estimator_
print("  ------------------------------------  ")
print (MethodID+"-1) BEST Configuration ("+modelName+") is  ==== ",best_grid )
print("  ------------------------------------  ")


# Prédire les valeurs en utilisant le : best_model
Y_pred_RF = best_model_searchRF.predict(X_test)
Y=y_test
num_data = X.shape[0]


# Erreurs pour l'évaluation des performances
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

mse = mean_squared_error(Y,Y_pred_RF)
rmse = math.sqrt(mse)
rse = math.sqrt(mse/(num_data-2))
mae=mean_absolute_error(Y,Y_pred_RF)

print(MethodID+"-2) Evaluation  "+modelName+" Results : ")
print("--> RMSE (("+modelName+"))=",rmse)
print("--> MAE (("+modelName+"))=",mae)
r = scipy.stats.pearsonr(Y,Y_pred_RF)
print("--> Pearson Correlation (("+modelName+"))=",r)


# Exporter le modèle
import pickle
filename = 'bestModel'+modelName+'-pluvio-vdp-nrml-Ecoli.pickle'
pickle.dump(best_model_searchRF, open(filename, 'wb'))
print("  ------------------------------------  ")
print( MethodID+"-3) Model " ,modelName," is generated and exported -- OK --  ")
print( " ################# ------------- ########################## ")

# Destandardisation des valeurs prédites
Y_pred_RF = (Y_pred_RF*y_std)+ y_mean
DataToExport["RF_Ecoli_pred"] = Y_pred_RF





In [ ]:

##############################################################################################"
######################################## MODEL DT  Split  ####################################"
####################################### Arbre de decision ####################################"

from sklearn.ensemble import RandomForestRegressor
## --- Local Params
modelName = "DT"
MethodID = "C"

print( " ################# ------------- ########################## ")
print( "Model " ,modelName," is runing ........ !  ")
from sklearn.neighbors import KNeighborsRegressor


from sklearn import tree
param_gridDT = {"max_depth": [1, 2, 10, 100],
              "random_state":[1, 2, 10, 100],
              "min_samples_leaf":[1, 2, 10, 100]
         }


# Sélection du meilleur modèle
modelDT =  tree.DecisionTreeRegressor()
best_model_searchDT = GridSearchCV(estimator = modelDT, param_grid = param_gridDT, cv = CROSS_VALID,
                           verbose = VERBOSE_VALUE)
best_model_searchDT.fit(X_train,y_train)


#  Montrer quel est le meilleur modèle
best_grid = best_model_searchDT.best_estimator_
print("  ------------------------------------  ")
print (MethodID+"-1) BEST Configuration ("+modelName+") is  ==== ",best_grid )
print("  ------------------------------------  ")


# Prédire les valeurs en utilisant best_model
Y_pred_DT = best_model_searchDT.predict(X_test)
Y=y_test
num_data = X.shape[0]


# erreurs pour l'évaluation des performances
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

mse = mean_squared_error(Y,Y_pred_DT)
rmse = math.sqrt(mse)
rse = math.sqrt(mse/(num_data-2))
mae=mean_absolute_error(Y,Y_pred_DT)

print(MethodID+"-2) Evaluation  "+modelName+" Results : ")
print("--> RMSE (("+modelName+"))=",rmse)
print("--> MAE (("+modelName+"))=",mae)
r = scipy.stats.pearsonr(Y,Y_pred_DT)
print("--> Pearson Correlation (("+modelName+"))=",r)


# Exporter le modèle
import pickle
filename = 'bestModel'+modelName+'-pluvio-vdp-nrml-Ecoli.pickle'
pickle.dump(best_model_searchDT, open(filename, 'wb'))
print("  ------------------------------------  ")
print( MethodID+"-3) Model " ,modelName," is generated and exported -- OK --  ")
print( " ################# ------------- ########################## ")

# Déstandardisation des valeurs prédites
Y_pred_DT = (Y_pred_DT*y_std)+ y_mean
DataToExport["DT_Ecoli_pred"] = Y_pred_DT


In [ ]:
##############################################################################################"
######################################## MODEL SVM  Split  ####################################"
################################ Machine a vecteurs de support  ###############################"
######################################### non lineaire ########################################"
 
from sklearn.ensemble import RandomForestRegressor
## --- Local Params
modelName = "SVM"
MethodID = "D"

print( " ################# ------------- ########################## ")
print( "Model " ,modelName," is runing ........ !  ")
from sklearn.neighbors import KNeighborsRegressor


from sklearn.svm import SVR
param_gridSVM = {'kernel' : ('sigmoid', 'rbf'),
         'coef0' : [0.01,10,0.5],
         'gamma' : ('auto','scale')
         }


# Sélection du meilleur modèle
modelSVM = SVR()
best_model_searchSVM = GridSearchCV(estimator = modelSVM, param_grid = param_gridSVM, cv = CROSS_VALID,
                           verbose = VERBOSE_VALUE)
best_model_searchSVM.fit(X_train,y_train)

#  Show which is the best model
best_grid = best_model_searchSVM.best_estimator_
print("  ------------------------------------  ")
print (MethodID+"-1) BEST Configuration ("+modelName+") is  ==== ",best_grid )
print("  ------------------------------------  ")

#  Montrer quel est le meilleur modèle
Y_pred_SVM = best_model_searchSVM.predict(X_test)
Y=y_test
num_data = X.shape[0]


# Prédire les valeurs en utilisant best_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

mse = mean_squared_error(Y,Y_pred_SVM)
rmse = math.sqrt(mse)
rse = math.sqrt(mse/(num_data-2))
mae=mean_absolute_error(Y,Y_pred_SVM)

print(MethodID+"-2) Evaluation  "+modelName+" Results : ")
print("--> RMSE (("+modelName+"))=",rmse)
print("--> MAE (("+modelName+"))=",mae)
r = scipy.stats.pearsonr(Y,Y_pred_SVM)
print("--> Pearson Correlation (("+modelName+"))=",r)


# Export the model
import pickle
filename = 'bestModel'+modelName+'-pluvio-vdp-nrml-Ecoli.pickle'
pickle.dump(best_model_searchSVM, open(filename, 'wb'))
print("  ------------------------------------  ")
print( MethodID+"-3) Model " ,modelName," is generated and exported -- OK --  ")
print( " ################# ------------- ########################## ")


# Destandardisation des valeurs prédites 
Y_pred_SVM = (Y_pred_SVM*y_std)+ y_mean
DataToExport["SVM_Ecoli_pred"] = Y_pred_SVM

In [ ]:
##############################################################################################"
######################################## MODEL AdaBoost  Split  ####################################"
##############################################################################################"

from sklearn.ensemble import RandomForestRegressor
## --- Local Params
modelName = "AdaBoost"
MethodID = "E"

print( " ################# ------------- ########################## ")
print( "Model " ,modelName," is runing ........ !  ")


from sklearn.ensemble import AdaBoostRegressor
param_gridADaBoost = {'base_estimator__min_samples_leaf':[1,2, 5],
              "learning_rate": [0.2,0.5],
              "n_estimators": [20, 50, 100]
         }


# Sélection du meilleur modèle
from sklearn import tree
DTC = tree.DecisionTreeRegressor(random_state = 11, max_features = "auto",
                                 max_depth = None)
ModelAdaBoostDTC = AdaBoostRegressor(base_estimator = DTC)
best_model_searchAdaBoost = GridSearchCV(estimator = ModelAdaBoostDTC, param_grid = param_gridADaBoost, cv = CROSS_VALID,
                           verbose = VERBOSE_VALUE)
best_model_searchAdaBoost.fit(X_train,y_train)


#  Montrer quel est le meilleur modèle
best_grid = best_model_searchAdaBoost.best_estimator_
print("  ------------------------------------  ")
print (MethodID+"-1) BEST Configuration ("+modelName+") is  ==== ",best_grid )
print("  ------------------------------------  ")


# Prédire les valeurs en utilisant best_model
Y_pred_AdaBoost = best_model_searchAdaBoost.predict(X_test)

Y=y_test
num_data = X.shape[0]


# Erreurs pour l'évaluation des performances
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

mse = mean_squared_error(Y,Y_pred_AdaBoost)
rmse = math.sqrt(mse)
rse = math.sqrt(mse/(num_data-2))
mae=mean_absolute_error(Y,Y_pred_AdaBoost)

print(MethodID+"-2) Evaluation  "+modelName+" Results : ")
print("--> RMSE (("+modelName+"))=",rmse)
print("--> MAE (("+modelName+"))=",mae)
r = scipy.stats.pearsonr(Y,Y_pred_AdaBoost)
print("--> Pearson Correlation (("+modelName+"))=",r)


# Exportez le modèle
import pickle
filename = 'bestModel'+modelName+'-pluvio-vdp-nrml-Ecoli.pickle'
pickle.dump(best_model_searchAdaBoost, open(filename, 'wb'))
print("  ------------------------------------  ")
print( MethodID+"-3) Model " ,modelName," is generated and exported -- OK --  ")
print( " ################# ------------- ########################## ")


# Destandardisation des valeurs prédites
Y_pred_AdaBoost = (Y_pred_AdaBoost*y_std)+ y_mean
DataToExport["AdaBoost_Ecoli_pred"] = Y_pred_AdaBoost





In [ ]:
##############################################################################################"
######################################## MODEL Bagging avec estimator RF ####################################"
##############################################################################################"

from sklearn.ensemble import RandomForestRegressor
## --- Local Params
modelName = "BaggingRF"
MethodID = "H"

print( " ################# ------------- ########################## ")
print( "Model " ,modelName," is runing ........ !  ")


from sklearn.ensemble import BaggingRegressor
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=100, n_features=7,n_informative=2, n_targets=1, random_state=0, shuffle=False)

param_gridBag = {
    'bootstrap': [True],
    'max_features': [0, 7],
    'n_estimators': [10, 50, 200]
}


# Sélection du meilleur modèle
modelRF =  RandomForestRegressor()
modelBaggingRF = BaggingRegressor(base_estimator=modelRF, n_estimators=10, random_state=0).fit(X, y) # base estimator par defaut DesicionTreeRegressor
best_model_searchBag2 = GridSearchCV(estimator = modelBaggingRF, param_grid = param_gridBag, cv = CROSS_VALID,
                           verbose = VERBOSE_VALUE)
best_model_searchBag2.fit(X_train,y_train)


#  Montrer quel est le meilleur modèle
best_grid = best_model_searchBag2.best_estimator_
print("  ------------------------------------  ")
print (MethodID+"-1) BEST Configuration ("+modelName+") is  ==== ",best_grid )
print("  ------------------------------------  ")


# Prédire les valeurs en utilisant best_model
Y_pred_Bag2 = best_model_searchBag2.predict(X_test)
Y=y_test
num_data = X.shape[0]


# Erreurs pour l'évaluation des performances
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

mse = mean_squared_error(Y,Y_pred_Bag2)
rmse = math.sqrt(mse)
rse = math.sqrt(mse/(num_data-2))
mae=mean_absolute_error(Y,Y_pred_Bag2)

print(MethodID+"-2) Evaluation  "+modelName+" Results : ")
print("--> RMSE (("+modelName+"))=",rmse)
print("--> MAE (("+modelName+"))=",mae)
r = scipy.stats.pearsonr(Y,Y_pred_Bag2)
print("--> Pearson Correlation (("+modelName+"))=",r)


# Exportez le modèle
import pickle
filename = 'bestModel'+modelName+'-pluvio-vdp-nrml-Ecoli.pickle'
pickle.dump(best_model_searchBag2, open(filename, 'wb'))
print("  ------------------------------------  ")
print( MethodID+"-3) Model " ,modelName," is generated and exported -- OK --  ")
print( " ################# ------------- ########################## ")


# Destandardisation des valeurs prédites
Y_pred_Bag2 = (Y_pred_Bag2*y_std)+ y_mean
DataToExport["Bagging_Ecoli_pred"] = Y_pred_Bag2

In [ ]:
# Les données à exporter
DataToExport

In [ ]:
######################################## prédictions pour la Marne ########################################################"

# Lire les données de la Marne
data2 = pd.read_csv("/home/manel/Bureau/BD/donnee+pluvio/smv+vdp-pluvio/donnee-sans-na/smv-vdp/smv2.csv")
X_test2 = data2.drop(["Ecoli","EI"],axis=1)
y_test2 = data2["Ecoli"]
X_test2 = X_test2.drop(["date"],axis=1)

# Standardisation des données de la seine 
X_test2 = (X_test2 - train_mean) / train_std
y_test2 = (y_test2 - y_mean) / y_std

# Importé les différents modèles générés précédemment
import pickle
loaded_model_knn=pickle.load(open('bestModelKNN-pluvio-vdp-nrml-Ecoli.pickle', 'rb'))
loaded_model_rf=pickle.load(open('bestModelRF-pluvio-vdp-nrml-Ecoli.pickle', 'rb'))
loaded_model_dt=pickle.load(open('bestModelDT-pluvio-vdp-nrml-Ecoli.pickle', 'rb'))
loaded_model_svm=pickle.load(open('bestModelSVM-pluvio-vdp-nrml-Ecoli.pickle', 'rb'))
loaded_model_adaboost=pickle.load(open('bestModelAdaBoost-pluvio-vdp-nrml-Ecoli.pickle', 'rb'))
loaded_model_bagging=pickle.load(open('bestModelBaggingRF-pluvio-vdp-nrml-Ecoli.pickle', 'rb'))


### Prédiction avec le modèle KNN
Y_pred_KNN = loaded_model_knn.predict(X_test2)

# Erreurs pour l'évaluation des performances
mse = mean_squared_error(y_test2,Y_pred_KNN)
rmse = math.sqrt(mse)
print('knn')
print("--> RMSE knn=",rmse)
mae=mean_absolute_error(y_test2,Y_pred_KNN)
print("--> MAE knn=",mae)
r = scipy.stats.pearsonr(y_test2,Y_pred_KNN)
print("--> Pearson Correlation knn=",r)

# Destandardisation des valeurs prédites
Y_pred_KNN = (Y_pred_KNN*y_std)+ y_mean
data2["knn"] = Y_pred_KNN


### Prédiction avec le modèle RF
Y_pred_rf = loaded_model_rf.predict(X_test2)

# Erreurs pour l'évaluation des performances
mse = mean_squared_error(y_test2,Y_pred_rf)
rmse = math.sqrt(mse)
print('rf')
print("--> RMSE rf=",rmse)
mae=mean_absolute_error(y_test2,Y_pred_rf)
print("--> MAE rf=",mae)
r = scipy.stats.pearsonr(y_test2,Y_pred_rf)
print("--> Pearson Correlation rf=",r)

# Destandardisation des valeurs prédites
Y_pred_rf = (Y_pred_rf*y_std)+ y_mean
data2["rf"] = Y_pred_rf


### Prédiction avec le modèle DT
Y_pred_dt = loaded_model_dt.predict(X_test2)

# Erreurs pour l'évaluation des performances
mse = mean_squared_error(y_test2,Y_pred_dt)
rmse = math.sqrt(mse)
print('dt')
print("--> RMSE dt=",rmse)
mae=mean_absolute_error(y_test2,Y_pred_dt)
print("--> MAE dt=",mae)
r = scipy.stats.pearsonr(y_test2,Y_pred_dt)
print("--> Pearson Correlation dt=",r)

# Destandardisation des valeurs prédites
Y_pred_dt = (Y_pred_dt*y_std)+ y_mean
data2["dt"] = Y_pred_dt


### Prédiction avec le modèle SVM
Y_pred_svm = loaded_model_svm.predict(X_test2)

# Erreurs pour l'évaluation des performances
mse = mean_squared_error(y_test2,Y_pred_svm)
rmse = math.sqrt(mse)
print('svm')
print("--> RMSE svm =",rmse)
mae=mean_absolute_error(y_test2,Y_pred_svm)
print("--> MAE svm=",mae)
r = scipy.stats.pearsonr(y_test2,Y_pred_svm)
print("--> Pearson Correlation svm=",r)

# Destandardisation des valeurs prédites
Y_pred_svm = (Y_pred_svm*y_std)+ y_mean
data2["svm"] = Y_pred_svm


### Prédiction avec le modèle AdaBoost
Y_pred_adaboost = loaded_model_adaboost.predict(X_test2)

# Erreurs pour l'évaluation des performances
mse = mean_squared_error(y_test2,Y_pred_adaboost)
rmse = math.sqrt(mse)
print('adaboost')
print("--> RMSE  adaboost =",rmse)
mae=mean_absolute_error(y_test2,Y_pred_adaboost)
print("--> MAE adaboost=",mae)
r = scipy.stats.pearsonr(y_test2,Y_pred_adaboost)
print("--> Pearson Correlation adaboost=",r)

# Destandardisation des valeurs prédites
Y_pred_adaboost = (Y_pred_adaboost*y_std)+ y_mean
data2["adaboost"] = Y_pred_adaboost


### Prédiction avec le modèle Bagging
Y_pred_bagging = loaded_model_bagging.predict(X_test2)

# Erreurs pour l'évaluation des performances
mse = mean_squared_error(y_test2,Y_pred_bagging)
rmse = math.sqrt(mse)
print('adaboost')
print("--> RMSE  bagging =",rmse)
mae=mean_absolute_error(y_test2,Y_pred_bagging)
print("--> MAE bagging=",mae)
r = scipy.stats.pearsonr(y_test2,Y_pred_bagging)
print("--> Pearson Correlation bagging=",r)

# Destandardisation des valeurs prédites
Y_pred_bagging = (Y_pred_bagging*y_std)+ y_mean
data2["bagging"] = Y_pred_bagging



In [ ]:
# Détermination de l'écart-type sur les données de la Marne.
import numpy as np
sd=np.std(y_test2)
print(sd)

In [ ]:
######################################## Exporter les données   ####################################"

# Exporter les données avec les valeurs de prédiction pour tous les modèles 
print( " ======================================================= ")
print( " ======================================================= ")

print("  * * * * * * * IV) Export CSV  * * * * * * *  ")
DataToExport.to_csv ('/home/manel/Bureau/BD/donnee+pluvio/smv+vdp-pluvio/donnee-sans-na/smv-vdp/ML/train_vdp/data_prediValuesML_ecoli_seine_seed0.csv', index = False, header=True, sep=";")
data2.to_csv ('/home/manel/Bureau/BD/donnee+pluvio/smv+vdp-pluvio/donnee-sans-na/smv-vdp/ML/train_vdp/data_prediValuesML_ecoli_smv_seed0.csv', index = False, header=True, sep=";")



In [ ]:
##############################################################################################"
################################### Analyser le modèle RF  ####################################"
##############################################################################################"

#! pip install treeinterpreter
from treeinterpreter import treeinterpreter as ti

rf = RandomForestRegressor(max_features=8, n_estimators=200)
rf.fit(X_train,y_train)

prediction, bias, contributions = ti.predict(rf, X_test)


In [ ]:
prediction, bias, contributions